<a href="https://colab.research.google.com/github/martishaaddams/mode_superpostion/blob/main/harmonical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import codecs
import numpy as np

def readData(filename):
    file = codecs.open(filename, mode='r', encoding='utf-8', errors='ignore')
    data = file.read()
    start_ver = data.find('NODE')
    end_ver = data.find('ELEMENT_SHELL')
    start_el = end_ver
    end_el = data.find('END')
    ver_strings = data[start_ver: end_ver].split('\n')[2:-4]
    el_strings = data[start_el: end_el].split('\n')[1:-1]
    ver_tmp = []
    el_tmp = []
    for element in ver_strings:
        tmp = element.split()
        ver_tmp.append([int(tmp[0]) - 1, float(tmp[1]), float(tmp[2])])
    for element in el_strings:
        tmp = element.split()
        el_tmp.append([int(tmp[0]) - 1, int(tmp[2]) - 1, int(tmp[3]) - 1, int(tmp[4]) - 1])
    ver_res = np.zeros((len(ver_strings), 2))
    el_res = np.zeros((len(ver_strings) + len(el_strings), 3), dtype='int')
    for element in ver_tmp:
        ver_res[element[0]][0], ver_res[element[0]][1] = element[1], element[2]
    for element in el_tmp:
        el_res[element[0]][0], el_res[element[0]][1], el_res[element[0]][2] = element[1], element[2], element[3]
    el_result = []
    for element in el_res:
        if not (element[0] == 0 and element[1] == 0 and element[2] == 0):
            el_result.append(element)

    return ver_res, np.array(el_result)
def Readnodes(filename):
    with codecs.open(filename, mode='r', encoding='utf-8', errors='ignore') as file:
        data = file.read()
    start, end = data.find('NODE'), data.find('ELEMENT_SHELL')
    raw_verts = data[start:end].split('\n')[2:-4]
    tmp = []
    max=0
    min=0
    for vert in raw_verts:
        args = vert.split()
        tmp.append([int(args[0])-1, float(args[1]), float(args[2])])
        if(max<=int(args[0])-1):
            max=int(args[0])-1
        if(min==0):
            min=int(args[0])-1
        if(min>=int(args[0])-1):
            min=int(args[0])-1
    #verts = np.zeros((max+1,2))
    verts = np.zeros((len(tmp),2))
    print(len(tmp))
    for elem in tmp:
        verts[elem[0]-min][0]=elem[1]
        verts[elem[0]-min][1]=elem[2]
    return verts

    
def Readelements(filename):
    with codecs.open(filename, mode='r', encoding='utf-8',errors='ignore') as file:
        data = file.read()
    start, end = data.find('ELEMENT_SHELL'), data.find('END')
    raw_verts= data[start:end].split('\n')[1:-1]
    tmp = []
    min_n=0
    for vert in raw_verts:
        args = vert.split()
        tmp.append([int(args[0])-1, int(args[2])-1, int(args[3])-1, int(args[4])-1])
        if(min_n==0):
            min_n=min(int(args[2])-1, int(args[3])-1, int(args[4])-1)
        else:
            min_n=min(min_n,int(args[2])-1, int(args[3])-1, int(args[4])-1)
    #print(tmp)
    
    triag = np.zeros((len(tmp),3), dtype = 'int')
    #print(triag.shape)
    for elem in tmp:
        triag[elem[0]][0] = elem[1]-min_n
        triag[elem[0]][1] = elem[2]-min_n
        triag[elem[0]][2] = elem[3]-min_n
    return triag


In [ ]:
from scipy.sparse import lil_matrix
from scipy.sparse import linalg
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

def get_physical_parameters():
    E = 2e10
    nu = 0.25
    rho = 7800
    #n = 10
    return E, nu, rho#, n


def get_U_result(ver, el, U):
    U_result = []
    n_vers = ver.shape[0]
    for j in range(0, 2 * n_vers - 1, 2):
        U_result.append(U[j]**2+U[j+1]**2)
    return U_result


def get_global_K(ver, el, E, m):
    n_vers = ver.shape[0]
    n_elems = el.shape[0]
    global_K_matrix = lil_matrix(
        (n_vers * 2, n_vers * 2))
    res = np.array([[1, m, 0], [m, 1, 0], [0, 0, (1 - m) / 2]])
    res = E / (1 - m ** 2) * res
    D_matrix = res
    for i in range(n_elems):
        x1, x2, x3 = ver[el[i][0]][0], ver[el[i][1]][0], ver[el[i][2]][0]
        y1, y2, y3 = ver[el[i][0]][1], ver[el[i][1]][1], ver[el[i][2]][1]
        mat = np.array([[x1, y1, 1], [x2, y2, 1], [x3, y3, 1]])
        area = get_area(mat)
        B_matrix = 1 / (2 * area) * np.array(
            [[y2 - y3, 0, y3 - y1, 0, y1 - y2, 0], [0, x3 - x2, 0, x1 - x3, 0, x2 - x1],
             [x3 - x2, y2 - y3, x1 - x3, y3 - y1, x2 - x1, y1 - y2]])
        B_matrix = np.array(B_matrix)
        local_K_matrix = np.dot(np.dot(B_matrix.T, D_matrix), B_matrix) * get_area(mat)
        for j in range(6):
            for s in range(6):
                start = 2 * el[i][j // 2] + j % 2
                end = 2 * el[i][s // 2] + s % 2
                global_K_matrix[start, end] += local_K_matrix[j, s]
    return global_K_matrix


def get_area(matrix):
    return np.abs(1 / 2 * np.linalg.det(matrix))

def get_global_M(ver, el, E, m, rho):
    n_vers = ver.shape[0]
    n_elems = el.shape[0]
    global_M_matrix = lil_matrix(
        (n_vers * 2, n_vers * 2))
    for i in range(n_elems):
        x1, x2, x3 = ver[el[i][0]][0], ver[el[i][1]][0], ver[el[i][2]][0]
        y1, y2, y3 = ver[el[i][0]][1], ver[el[i][1]][1], ver[el[i][2]][1]
        mat = np.array([[x1, y1, 1], [x2, y2, 1], [x3, y3, 1]])
        area = get_area(mat)
        m = rho * area
        local_M_matrix = np.array([[2, 0, 1, 0, 1, 0],
                         [0, 2, 0, 1, 0, 1],
                         [1, 0, 2, 0, 1, 0],
                         [0, 1, 0, 2, 0, 1],
                         [1, 0, 1, 0, 2, 0],
                         [0, 1, 0, 1, 0, 2],]) *m / 12
        
        for j in range(6):
            for s in range(6):
                start = 2 * el[i][j // 2] + j % 2
                end = 2 * el[i][s // 2] + s % 2
                global_M_matrix[start, end] += local_M_matrix[j, s]
    return global_M_matrix


def get_eigenvalues(K, M, n):
   
    values, vectors =  linalg.eigsh(K, n, M, which ='SM')

    return values, vectors

def get_mj(M, u):
    m = M @ u
    s = u.T@m
    return s

def interpolation(x, y, nodes, elements, U_sum):
    u=0
    n_elems = elements.shape[0]
    for i in range(n_elems):
        if is_in_element(nodes, elements, i, x, y):
            u = (U_sum[elements[i][0]]+U_sum[elements[i][1]]+U_sum[elements[i][2]])/3
            return u

def get_U(ver, el, eigen_vectors, mod):
    U = []
    n_vers = ver.shape[0]
    for j in range(2 * n_vers):
        U.append(eigen_vectors[j, mod])
    print(U)
    return U

def is_in_element(verts, element, i, x4, y4, eps = 0.000001):
    el0, el1, el2 = element[i][0], element[i][1], element[i][2]
    x1, y1 = verts[el0][0], verts[el0][1]
    x2, y2 = verts[el1][0], verts[el1][1]
    x3, y3 = verts[el2][0], verts[el2][1]
    S0 = element_area_get(x1, y1, x2, y2, x3, y3)
    S1 = element_area_get(x4, y4, x2, y2, x3, y3)
    S2 = element_area_get(x1, y1, x4, y4, x3, y3)
    S3 = element_area_get(x1, y1, x2, y2, x4, y4)
    if abs(S0 - S1 - S2 -S3) < eps:
        return True
    return False

def element_area_get(x1, y1, x2, y2, x3, y3):
    cur_det = np.array([[x1, y1, 1], [x2, y2, 1], [x3, y3, 1]])
    elem_area = np.abs(1/2*np.linalg.det(cur_det))
    return elem_area


def get_normal_vec(ver, data):
    norm_vec = []
    x_coord = 0
    y_coord = 1
    n_elems = data.shape[0]
    for i_elem in range(n_elems):
        node_num1, node_num2, node_num3 = data[i_elem][1], data[i_elem][2], data[i_elem][3]
        x_tmp = ver[node_num2][y_coord] - ver[node_num1][y_coord]
        y_tmp = ver[node_num1][x_coord] - ver[node_num2][x_coord]
        len_el = np.sqrt(x_tmp ** 2 + y_tmp ** 2)
        x_norm, y_norm = x_tmp / len_el, y_tmp / len_el
        tmp_vec_x = ver[node_num2][x_coord] - ver[node_num3][x_coord]
        tmp_vec_y = ver[node_num2][y_coord] - ver[node_num3][y_coord]
        if tmp_vec_x * x_norm + tmp_vec_y * y_norm < 0:
            x_norm, y_norm = -x_norm, -y_norm
        norm_vec.append([x_norm, y_norm])
    return np.array(norm_vec)

from math import sin

def get_F(F,ver, upBorder_nodes, t, w, A):
    x_coord = 0
    y_coord = 1
    n_up = upBorder_nodes.shape[0]
    n_vers = ver.shape[0]
    F = lil_matrix((2 * n_vers, 1))
    norm_vec = get_normal_vec(ver, upBorder_nodes)
    for i_ver in range(n_up):
        num_node1, num_node2 = upBorder_nodes[i_ver][1], upBorder_nodes[i_ver][2]
        x_dif = ver[num_node1][x_coord] - ver[num_node2][x_coord]
        y_dif = ver[num_node1][y_coord] - ver[num_node2][y_coord]
        len_el = np.sqrt(x_dif ** 2 + y_dif ** 2) / 2
        F[2 * num_node1, 0] += (A*sin(w*t) * norm_vec[i_ver][0] * len_el)
        F[2 * num_node2, 0] += (A*sin(w*t) * norm_vec[i_ver][0] * len_el)
        F[2 * num_node1 + 1, 0] += (A*sin(w*t) * norm_vec[i_ver][1] * len_el)
        F[2 * num_node2 + 1, 0] += (A*sin(w*t) * norm_vec[i_ver][1] * len_el)
        #F[2 * num_node1, 0] += (P * norm_vec[i_ver][0] * len_el)
        #F[2 * num_node2, 0] += (P * norm_vec[i_ver][0] * len_el)
        #F[2 * num_node1 + 1, 0] += (P * norm_vec[i_ver][1] * len_el)
        #F[2 * num_node2 + 1, 0] += (P * norm_vec[i_ver][1] * len_el)
    return F

def get_elements_on_borders(ver, el, x1, y1_min, y1_max,
          x2, y2_min, y2_max):
    right_nodes, left_nodes = [], []
    n_elems = el.shape[0]
    for i_elem in range(n_elems):
        not_on_bord_r = 3
        not_on_bord_l = 3
        for node in el[i_elem]:
            if np.isclose(ver[node][0], x1) and ver[node][1]<=y1_max and ver[node][1]>=y1_min:
                not_on_bord_r -= 1
            if np.isclose(ver[node][0], x2) and ver[node][1]<=y2_max and ver[node][1]>=y2_min:
                not_on_bord_l -= 1
        if not_on_bord_r == 1:
            res_r = [i_elem]
            in_node = 0
            for node in el[i_elem]:
                if np.isclose(ver[node][0], x1) and ver[node][1]<=y1_max and ver[node][1]>=y1_min:
                    res_r.append(node)
                else:
                    in_node = node
            res_r.append(in_node)
            right_nodes.append(res_r)
        if not_on_bord_l == 1:
            res_l = [i_elem]
            in_node = 0
            for node in el[i_elem]:
                if np.isclose(ver[node][0], x2) and ver[node][1]<=y2_max and ver[node][1]>=y2_min:
                    res_l.append(node)
                else:
                    in_node = node
            res_l.append(in_node)
            left_nodes.append(res_l)
    right_nodes = np.array(right_nodes).astype(int)
    left_nodes = np.array(left_nodes).astype(int)
    return right_nodes, left_nodes

def show_borders(ver, el, circ1, circ2):
    plt.figure(figsize=(9, 9), facecolor='pink')
    plt.triplot(ver[:, 0], ver[:, 1], el)
    plt.triplot(ver[:, 0], ver[:, 1], el[circ1[:, 0]], color='red')
    plt.triplot(ver[:, 0], ver[:, 1], el[circ2[:, 0]], color='blue')
    plt.show()

In [ ]:
def GetNi(verts, data):    
    n_vec = []
    #print(data.shape)
    for i in range(data.shape[0]):
        nx = verts[data[i][2]][1] - verts[data[i][1]][1]
        ny = verts[data[i][1]][0] - verts[data[i][2]][0]
        nx = nx/np.sqrt(nx**2+ny**2)
        ny = ny/np.sqrt(nx**2+ny**2)
        test_x = verts[data[i][2]][0] - verts[data[i][3]][0]
        test_y = verts[data[i][2]][1] - verts[data[i][3]][1]
        if test_x*nx +test_y*ny < 0:
            nx = -nx
            ny = -ny
        n_vec.append([nx, ny])
    n_vec = np.array(n_vec)
    #print(n_vec)
    #print("next")
    return n_vec
def GetGlobalF(verts, data, global_F,time,omega,ampl,c=True):
  n_vec = GetNi(verts, data)
  P=ampl*sin(omega*time)
  for i in range(data.shape[0]):
      l = np.sqrt(np.sum((verts[data[i][1]]- verts[data[i][2]])**2))/2
      if c:
          global_F[2*data[i][1], 0] += (P*n_vec[i][0]*l)
          global_F[2*data[i][1]+1, 0] += (P*n_vec[i][1]*l)
          global_F[2*data[i][2], 0] += (P*n_vec[i][0]*l)
          global_F[2*data[i][2]+1, 0] += (P*n_vec[i][1]*l)
      else:
          global_F[2*data[i][1], 0] += (P*n_vec[i][1]*l)
          global_F[2*data[i][1]+1, 0] += (P*n_vec[i][0]*l)
          global_F[2*data[i][2], 0] += (P*n_vec[i][1]*l)
          global_F[2*data[i][2]+1, 0] += (P*n_vec[i][0]*l)
  return global_F

In [ ]:
def adjacency_matrix(nodes,elements):
    #triangle shape
    sc=nodes.shape[0]
    # adj_m = np.zeros((sc,sc))
    adj_m = sparse.lil_matrix((sc, sc))
    for element in elements:
        adj_m[element[0],element[1]]+=1
        adj_m[element[1], element[0]]+=1
        adj_m[element[1],element[2]]+=1
        adj_m[element[2],element[1]]+=1
        adj_m[element[2],element[0]]+=1
        adj_m[element[0],element[2]]+=1
    return adj_m


def reorder_matrix(nodes, elements, adjacency):
    # Compute the Cuthill-McKee ordering of the nodes
    row, col = adjacency.nonzero()
    # adjacency_list = [list(col[row == i]) for i in range(nodes.shape[0])]
    graph = sparse.csr_matrix(adjacency)
    cuthill_mckee_ordering = list(sparse.csgraph.reverse_cuthill_mckee(graph))

    # Reorder the nodes and elements based on the Cuthill-McKee ordering
    nodes = nodes[cuthill_mckee_ordering]
    elements = np.array([[cuthill_mckee_ordering.index(el[0]),
                          cuthill_mckee_ordering.index(el[1]),
                          cuthill_mckee_ordering.index(el[2])] for el in elements])

    # Recompute the adjacency matrix based on the reordered nodes and elements
    adjacency = adjacency_matrix(nodes, elements)

    return adjacency, nodes, elements

def smborderelements(nodes,elements,ad_j):
    face=[]
    eyer=[]
    eyel=[]
    smile=[]
    bo=[]
    index=[]
    i=0
    eps=0.001
    for elem in elements:
        
        if  ad_j[elem[0],elem[1]]==1 or ad_j[elem[0],elem[2]]==1 or ad_j[elem[1],elem[2]]==1:
            bo.append(elem)
            index.append(i)
        i+=1
    for e in range(len(bo)):
        elem=bo[e]
        
        if ad_j[elem[0],elem[1]]==1:
            treug_pl_index=[index[e]]
            treug_pl_index.append(elem[0])
            treug_pl_index.append(elem[1])
            treug_pl_index.append(elem[2])
            if(abs(nodes[elem[0]][0]**2+nodes[elem[0]][1]**2-100)<=eps ):
                face.append(treug_pl_index)
            else:
                if(abs((nodes[elem[0]][0]-4)**2+(nodes[elem[0]][1]-3)**2-4)<=eps ):
                    eyer.append(treug_pl_index)
                else:
                    if(abs((nodes[elem[0]][0]+4)**2+(nodes[elem[0]][1]-3)**2-4)<=eps ):
                        eyel.append(treug_pl_index)
                    else:
                        smile.append(treug_pl_index)
        if ad_j[elem[0],elem[2]]==1:
            treug_pl_index=[index[e]]
            treug_pl_index.append(elem[0])
            treug_pl_index.append(elem[2])
            treug_pl_index.append(elem[1])
            
            if(abs(nodes[elem[0]][0]**2+nodes[elem[0]][1]**2-100)<=eps ):
                face.append(treug_pl_index)
            else:
                if(abs((nodes[elem[0]][0]-4)**2+(nodes[elem[0]][1]-3)**2-4)<=eps):
                    eyer.append(treug_pl_index)
                else:
                    if(abs((nodes[elem[0]][0]+4)**2+(nodes[elem[0]][1]-3)**2-4)<=eps ):
                        eyel.append(treug_pl_index)
                    else:
                        smile.append(treug_pl_index)
        if ad_j[elem[1],elem[2]]==1:
            treug_pl_index=[index[e]]
            treug_pl_index.append(elem[1])
            treug_pl_index.append(elem[2])
            treug_pl_index.append(elem[0])
            if(abs(nodes[elem[1]][0]**2+nodes[elem[1]][1]**2-100)<=eps ):
                face.append(treug_pl_index)
            else:
                if(abs((nodes[elem[1]][0]-4)**2+(nodes[elem[1]][1]-3)**2-4)<=eps ):
                    eyer.append(treug_pl_index)
                else:
                    if(abs((nodes[elem[1]][0]+4)**2+(nodes[elem[1]][1]-3)**2-4)<=eps ):
                        eyel.append(treug_pl_index)
                    else:
                        smile.append(treug_pl_index)
            
    face=np.array(face)
    eyer=np.array(eyer)
    eyel=np.array(eyel)
    smile=np.array(smile)
    #inb=np.array(inb)
    return face,eyer,eyel,smile

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

def show_mesh(filename):
    ver, el = readData(filename)
    fig = plt.figure(figsize = (15, 6), facecolor = 'pink')
    n_elems = el.shape[0]
    for i in range(n_elems):
            x1, x2, x3 = ver[el[i][0]][0], ver[el[i][1]][0], ver[el[i][2]][0]
            y1, y2, y3 = ver[el[i][0]][1], ver[el[i][1]][1], ver[el[i][2]][1]
            plt.plot([x1,x2,x3],[y1,y2,y3], color='b',
                marker='.',
                linestyle='dotted')
    plt.show()

def show_field(filename, mod, E, nu, rho, n):

    ver, el = readData(filename)
    K = get_global_K(ver, el, E, nu)
    M = get_global_M(ver, el, E, nu, rho)
    eigen_values, eigen_vectors = get_eigenvalues (K, M, n)
    size = (15, 5)
    fig = plt.figure(figsize = size)
    x = []
    y = []
    for i in range(ver.shape[0]):
        x.append(ver[i][0])
        y.append(ver[i][1])
    U = get_U(ver, el, eigen_vectors, mod)
    U_result = get_U_result(ver, el, U)
    plt.title('Номер моды ' + str(mod + 1), fontsize = 15)
    plt.tripcolor(x, y, el, U_result)
    plt.colorbar()
    plt.show()

def draw_line(mod, E, nu, rho, n):
    
    size = (12, 8)
    fig, ax = plt.subplots(figsize = size)
    for i in tqdm(range(1, 3)):
        filename = 'meshes1/m' + str(i) + '.k'
        ver, el = readData(filename)
        K = get_global_K(ver, el, E, nu)
        M = get_global_M(ver, el, E, nu, rho)
        eigen_values, eigen_vectors = get_eigenvalues (K, M, n)
        U = get_U(ver, el, eigen_vectors, mod)
        U_result = get_U_result(ver, el, U)
        y_line = 2.5
        x = np.linspace(0, 10, 100)
        y = []
        for j in range(x.shape[0]):
            s = interpolation(x[j], y_line, ver, el, U_result)
            y.append(s)
        ax.plot(x, y, label = 'mesh with ' + str(el.shape[0]) + ' elements')
        ax.legend(loc = 'upper left')
    ax.set_ylabel('Перемещения вдоль линии y = ' + str(y_line), fontsize = 15)
    plt.title('Номер моды ' + str(mod + 1), fontsize = 15)

    plt.show()

def draw_frequency(E, nu, rho, n):
    fig, ax = plt.subplots(figsize=(12, 6))
    for i in tqdm(range(1, 3)):
        filename = 'meshes1/m' + str(i) + '.k'
        ver, el = readData(filename)
        K = get_global_K(ver, el, E, nu)
        M = get_global_M(ver, el, E, nu, rho)
        eigen_values, eigen_vectors = get_eigenvalues (K, M, n)
        x = np.linspace(0, 10, 10, endpoint = False)
        y = ((abs(eigen_values)**0.5)/(2*np.pi))
        ax.plot(x, y)
    ax.set_xlabel('Мода', fontsize = 15)
    ax.set_ylabel(chr(969) + ', Hz', fontsize = 15)
    plt.title('Значения частот', fontsize = 15)
    plt.show()


def print_eigenvalues():
    for i in tqdm(range(1, 3)):
        print(1)
        E, nu, rho, n = get_physical_parameters()
        filename = 'meshes1/m' + str(i) + '.k'
        ver, el = readData(filename)
        print('mesh with ' + str(ver.shape[0]) + ' nodes and ' + str(el.shape[0]) + ' elements')
        K = get_global_K(ver, el, E, nu)
        M = get_global_M(ver, el, E, nu, rho)
        eigen_values, eigen_vectors = get_eigenvalues (K, M, n)
        for j in range(n):
            s = round((abs(eigen_values[j])**0.5)/(2*np.pi),3)
            print(chr(969) + str(j+1) + ' = ', s , ' Hz')
    


In [ ]:
import scipy.sparse as sparse
def ApplyBoundCondForF(global_K,global_F, data, moves):
  fixes = list(set(data[:,1]) | set(data[:,2]))
  for i in range(len(fixes)):
    if moves[0] is not None:
        x = 2*fixes[i]
        global_F[x,0] = moves[0]
        extra_F = sparse.lil_matrix(global_F.shape)
        extra_F[x,0] = moves[0]
        #rint(global_K[:,x].shape)
        #Sprint(extra_F.shape)
        e=np.transpose(extra_F)
        for j in range(global_K.shape[0]):
            el=global_K[j,x]*moves[0] 
            global_F[j,0]-=el#np.dot( global_K[:,x],extra_F)
        if moves[1] is not None:
            x = 2*fixes[i]+1
            global_F[x,0] = moves[1]
            extra_F = sparse.lil_matrix(global_F.shape)
            extra_F[x,0] = moves[1]
            #global_F -= np.dot(global_K, extra_F)
            for j in range(global_K.shape[0]):
                el=global_K[j,x]*moves[1] 
                global_F[j,0]-=el
  return global_F

def ApplyBoundCondN(global_K, global_F, data, moves):
    fixes = list(set(data[:,1]) | set(data[:,2]))
    for i in range(len(fixes)):
        if moves[0] is not None:
          x = 2*fixes[i]
          global_F[x,0] = moves[0]
          extra_F = sparse.lil_matrix(global_F.shape)
          extra_F[x,0] = moves[0]
          global_F -= np.dot(global_K, extra_F)
          temp = global_K[x,x]
          global_K[x,:] = 0
          global_K[:,x] = 0
          global_K[x,x] = temp
          x = 2*fixes[i]
          global_F[x,0] = moves[0]
          extra_F = sparse.lil_matrix(global_F.shape)
          extra_F[x,0] = moves[0]
          #rint(global_K[:,x].shape)
          #Sprint(extra_F.shape)
        if moves[1] is not None:
          x = 2*fixes[i]+1
          global_F[x,0] = moves[1]
          extra_F = sparse.lil_matrix(global_F.shape)
          extra_F[x,0] = moves[1]
          global_F -= np.dot(global_K, extra_F)
          temp = global_K[x,x]
          global_K[x,:] = 0
          global_K[:,x] = 0
          global_K[x,x] = temp
    return global_K, global_F

def GenerateD(E, nu):
    res = np.array([[1, nu, 0], [nu, 1, 0], [0, 0, (1-nu)/2]])
    res = E/(1-nu**2) * res
    return res
def GenerateB(verts, element, i):
    x1 = verts[element[i][0]][0]
    y1 = verts[element[i][0]][1]
    x2 = verts[element[i][1]][0]
    y2 = verts[element[i][1]][1]
    x3 = verts[element[i][2]][0]
    y3 = verts[element[i][2]][1]
    cur_det = np.array([[x1, y1, 1], [x2, y2, 1], [x3, y3, 1]])
    elem_area = np.abs(1/2*np.linalg.det(cur_det))
    B = 1/(2*elem_area)*np.array([[y2-y3, 0, y3-y1, 0, y1-y2, 0], 
                                  [0, x3-x2, 0, x1-x3, 0, x2-x1], 
                                  [x3-x2, y2-y3, x1-x3, y3-y1, x2-x1, y1-y2]])
    return B
def GetP(elements, nodes, U, E, nu):
    stress_data = np.zeros((elements.shape[0],3))
    deformation_data = np.zeros((elements.shape[0],3))
    D = GenerateD(E, nu)
    for i in range(elements.shape[0]):
        elem=elements[i]
        u_loc = np.zeros((6,1))
        u_loc[0] = U[elem[0]*2]
        u_loc[1] = U[elem[0]*2+1]
        u_loc[2] = U[elem[1]*2]
        u_loc[3] = U[elem[1]*2+1]
        u_loc[4] = U[elem[2]*2]
        u_loc[5] = U[elem[2]*2+1]
        B = GenerateB(nodes, elements, i)
        deformation = B.dot(u_loc)
        stress = D.dot(deformation)
        stress_data[i] = stress.reshape(1,-1)
        deformation_data[i] = deformation.reshape(1,-1)
    return stress_data, deformation_data

# New Section

<ipython-input-49-dff9f38cd368>:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-poster')


In [ ]:
t = np.linspace(0, 0.1, 101)
print(t)

[0.    0.001 0.002 0.003 0.004 0.005 0.006 0.007 0.008 0.009 0.01  0.011
 0.012 0.013 0.014 0.015 0.016 0.017 0.018 0.019 0.02  0.021 0.022 0.023
 0.024 0.025 0.026 0.027 0.028 0.029 0.03  0.031 0.032 0.033 0.034 0.035
 0.036 0.037 0.038 0.039 0.04  0.041 0.042 0.043 0.044 0.045 0.046 0.047
 0.048 0.049 0.05  0.051 0.052 0.053 0.054 0.055 0.056 0.057 0.058 0.059
 0.06  0.061 0.062 0.063 0.064 0.065 0.066 0.067 0.068 0.069 0.07  0.071
 0.072 0.073 0.074 0.075 0.076 0.077 0.078 0.079 0.08  0.081 0.082 0.083
 0.084 0.085 0.086 0.087 0.088 0.089 0.09  0.091 0.092 0.093 0.094 0.095
 0.096 0.097 0.098 0.099 0.1  ]


In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
from google.colab import files

plot_stress_xx = []
plot_stress_yy = []
plot_stress_xy = []

element_stress1=[]

filenames =['smile_1_6.k','smile_08.k','smile_04.k']#,'smile_02.k']
for filename in filenames:
  if filename =='smile_1_6.k':
    ver, el = readData(filename)
    
  else:
    ver=Readnodes(filename)
    el=Readelements(filename)
  print(el)

  fname=filename.split(".")[0]
  isExist=os.path.exists("./"+fname)
  if not isExist:
    os.mkdir("./"+fname)
  A = 100
  w = np.pi/2
  t1 = 1
  #right_border, left_border = get_elements_on_borders(ver, el, 10, 0, 4, 0, 0, 5)
  ad_j=adjacency_matrix(ver,el)
  ad_j,ver,el=reorder_matrix(ver,el,ad_j)
  face,eyer,eyel,smile=smborderelements(ver,el,ad_j)
  #print()
  F = lil_matrix((2*ver.shape[0], 1))
  F =GetGlobalF(ver,face,F, t1, w, A)
  #print(F)
  E, nu, rho = get_physical_parameters()

  n=int(ver.size/2)
  #n=40
  mod = 3
  K = get_global_K(ver, el, E, nu)
  M = get_global_M(ver, el, E, nu, rho)
  K, F = ApplyBoundCondN(K, F, smile, [0, None])
  isExist=os.path.exists("./"+fname+".npz")
  if not isExist:
    evals, evecs = get_eigenvalues (K, M, n)
    np.savez(fname, evals=evals,evecs=evecs)
    files.download(fname+".npz")
  else:
    npzfile = np.load(fname+'.npz')
    evals=npzfile['evals']
    evecs=npzfile['evecs']
    #files.download(fname+".npz")

  alpha = []
  mj = []
  #Fj = []
  wj = [((abs(i)**0.5)/(2*np.pi)) for i in evals]
  for j in range(n):
      mj.append(get_mj(M, np.asarray(evecs[:,j])))
      #Fj.append((F.T@evecs[:,j])[0])
      #alpha.append(Fj[j]/((wj[j]**2-w**2)*mj[j]))
  #U = []
  #for i in range(310):
    #U.append(evecs[i]@alpha)
  #print(U)
  #print(len(U))

  #show_borders(ver,el,right_border,left_border)
  #show_mesh(filename)
  #show_field(filename, mod, E, nu, rho, n)
  #draw_frequency(E, nu, rho, n)
  #draw_line(mod, E, nu, rho, n)
  #print_eigenvalues()
  OMEGA_F=100
  AMPL=10
  #import matplotlib.pyplot as plt
  #import numpy as np
  from scipy.integrate import solve_ivp

  plt.style.use('seaborn-poster')

  %matplotlib inline
  def alpha_find(y, t, om,fj,mj):
    alpha, diffalpha = y
    dydt = [diffalpha,fj/mj-om**2*alpha]
    return dydt
  y0 = [0.0, 0.0]

  t = np.linspace(0, 0.1, 201)
  tt1=np.linspace(0, 0.1, 401)
  print(t)

  from scipy.integrate import odeint
  alpha=np.zeros((n))
  ALPHA=np.zeros((t.shape[0],n))
  ALPHA1=np.zeros((tt1.shape[0],n))
  for tim in range(t.shape[0]-1):
    #F = sparse.lil_matrix((2*ver.shape[0], 1))
    print(t[tim])
    F = GetGlobalF(ver, face,F, t[tim], OMEGA_F, AMPL)
    print(F)
    F = ApplyBoundCondForF(K, F, smile, [0, None])
    print(F)
    liltime=5

    time=np.linspace(t[tim], t[tim+1], liltime)
    for j in range(n):
    # F.tocsr()
      #print(F)
      fj=F[j,0]
      print(fj)
      print(evals[j])
      sol = odeint(alpha_find, y0, time, args=(evals[j],fj,mj[j]))
      #for k in range (t.shape[0]):
      ALPHA[tim][j]=sol[liltime-1][0]
  # for tim in range(tt1.shape[0]-1):
  #   #F = sparse.lil_matrix((2*ver.shape[0], 1))
  #   print(tt1[tim])
  #   F = GetGlobalF(ver, face,F, tt1[tim], OMEGA_F, AMPL)
  #   print(F)
  #   F = ApplyBoundCondForF(K, F, smile, [0, None])
  #   print(F)
  #   liltime=10

  #   time=np.linspace(tt1[tim], tt1[tim+1], liltime)
  #   for j in range(n):
  #   # F.tocsr()
  #     #print(F)
  #     fj=F[j,0]
  #     print(fj)
  #     print(evals[j])
  #     sol = odeint(alpha_find, y0, time, args=(evals[j],fj,mj[j]))
  #     #for k in range (t.shape[0]):
  #     ALPHA1[tim][j]=sol[liltime-1][0]
  #     # print(sol)

  # for j in range (n):
  #   plt.figure()
  #   print(ALPHA[:,j].shape)
  #   plt.plot(t,ALPHA[:,j],'b')
  #   plt.plot(tt1,ALPHA1[:,j],'r--')
  #   #er1=error(ALPHA[:,j],ALPHA1[:,j])
  #   plt.title('alpha {}'.format(j))
  #   plt.savefig('./alpha/{}.png'.format(j))
  plt.close('all')
  x4, y4 = 0, 5
  for i in range(el.shape[0]):
    if (is_in_element(ver, el, i, x4, y4)):
      element_number = i
      break  
  element_stress = []
  for k in range(t.shape[0]):
    for j in range(n):
      alpha[j]=ALPHA[k][0]
      print(alpha[j])
    
    element_deformations = []
    print("You are a winner")
    U=[]
    for i in range(2*ver.shape[0]):
      U.append(evecs[i]@alpha)
    x = []
    y = []
    for i in range(ver.shape[0]):
      x.append(ver[i][0])
      y.append(ver[i][1])
    #print(len(U))
    U_result = get_U_result(ver, el, U)
    stress, deformations = GetP(el, ver, U, E, nu)
    element_stress.append(stress[element_number])
    element_deformations.append(deformations[element_number])
    #element_stress = np.asarray(element_stress)
    #plot_stress_xx=[stress]

    print(U[0].shape)
    plt.figure()
    plt.tripcolor(x, y, el,stress[:,0])
    #x = [point[0] for point in ver]
    #y = [point[1] for point in ver]
    m = 14
    N = len(ver)
    plt.title('stress {} {}'.format(t[k],(n+1)))
    #plt.tripcolor(x, y,el, U)
    plt.colorbar()
    plt.savefig('./{}/stress_{}_{}.png'.format(fname,t[k],(n+1)))
    plt.close('all')
  element_stress1.append( np.asarray(element_stress))
# plot_stress_xx.append(abs(element_stress[:,0]))
# plot_stress_yy.append(abs(element_stress[:,1]))
# plot_stress_xy.append(abs(element_stress[:,2]))

[[ 74  28   2]
 [ 75   5   6]
 [ 76   0   5]
 [ 77   6   7]
 [ 78   7   8]
 [ 69  79  68]
 [ 80   8   9]
 [ 22  81   2]
 [ 23  82  22]
 [ 83 157  23]
 [139  83  24]
 [ 84  12   1]
 [ 29  85   3]
 [ 86  11  12]
 [ 87  10  11]
 [ 84   1  88]
 [ 32  33  89]
 [ 90   9  10]
 [ 91 156  30]
 [ 32  92  31]
 [ 93  27  28]
 [ 34  94  33]
 [ 95   1  13]
 [ 96  35   3]
 [141  88   1]
 [ 36  97   4]
 [ 98  97  36]
 [ 99  34  35]
 [100  98  37]
 [ 39  96  38]
 [ 40  41  99]
 [101  96  39]
 [151  34  99]
 [103 102  42]
 [104  26  27]
 [152  33  94]
 [107 106  45]
 [108 107  46]
 [109  25  26]
 [110 108  47]
 [ 49 111  48]
 [ 51  52 109]
 [ 49 150 104]
 [ 51 150  50]
 [112  24  25]
 [113 109  52]
 [114  21   0]
 [115 113  53]
 [116  20  21]
 [117 115  54]
 [118  19  20]
 [119 117  55]
 [120  18  19]
 [121 144 119]
 [ 58 122  57]
 [123  17  18]
 [124 114 122]
 [125 124  59]
 [ 61 126  60]
 [120 126  61]
 [127  16  17]
 [128 120  62]
 [129  15  16]
 [ 64 123  63]
 [130  14  15]
 [127 123  64]
 [ 66 127 

<ipython-input-27-81a1e292bdfb>:83: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-poster')
/usr/local/lib/python3.10/dist-packages/scipy/integrate/_odepack_py.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


Streaming output truncated to the last 5000 lines.
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
4.833312649806312e-05
You are a winner
()
4.920723410346698e-05
4.920723410346698e-05
4.920

<ipython-input-27-81a1e292bdfb>:83: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-poster')


[0.     0.0005 0.001  0.0015 0.002  0.0025 0.003  0.0035 0.004  0.0045
 0.005  0.0055 0.006  0.0065 0.007  0.0075 0.008  0.0085 0.009  0.0095
 0.01   0.0105 0.011  0.0115 0.012  0.0125 0.013  0.0135 0.014  0.0145
 0.015  0.0155 0.016  0.0165 0.017  0.0175 0.018  0.0185 0.019  0.0195
 0.02   0.0205 0.021  0.0215 0.022  0.0225 0.023  0.0235 0.024  0.0245
 0.025  0.0255 0.026  0.0265 0.027  0.0275 0.028  0.0285 0.029  0.0295
 0.03   0.0305 0.031  0.0315 0.032  0.0325 0.033  0.0335 0.034  0.0345
 0.035  0.0355 0.036  0.0365 0.037  0.0375 0.038  0.0385 0.039  0.0395
 0.04   0.0405 0.041  0.0415 0.042  0.0425 0.043  0.0435 0.044  0.0445
 0.045  0.0455 0.046  0.0465 0.047  0.0475 0.048  0.0485 0.049  0.0495
 0.05   0.0505 0.051  0.0515 0.052  0.0525 0.053  0.0535 0.054  0.0545
 0.055  0.0555 0.056  0.0565 0.057  0.0575 0.058  0.0585 0.059  0.0595
 0.06   0.0605 0.061  0.0615 0.062  0.0625 0.063  0.0635 0.064  0.0645
 0.065  0.0655 0.066  0.0665 0.067  0.0675 0.068  0.0685 0.069  0.0695
 0.07 

...

Streaming output truncated to the last 5000 lines.
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859815524e-05
-1.2591300859

<ipython-input-27-81a1e292bdfb>:83: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-poster')


[0.     0.0005 0.001  0.0015 0.002  0.0025 0.003  0.0035 0.004  0.0045
 0.005  0.0055 0.006  0.0065 0.007  0.0075 0.008  0.0085 0.009  0.0095
 0.01   0.0105 0.011  0.0115 0.012  0.0125 0.013  0.0135 0.014  0.0145
 0.015  0.0155 0.016  0.0165 0.017  0.0175 0.018  0.0185 0.019  0.0195
 0.02   0.0205 0.021  0.0215 0.022  0.0225 0.023  0.0235 0.024  0.0245
 0.025  0.0255 0.026  0.0265 0.027  0.0275 0.028  0.0285 0.029  0.0295
 0.03   0.0305 0.031  0.0315 0.032  0.0325 0.033  0.0335 0.034  0.0345
 0.035  0.0355 0.036  0.0365 0.037  0.0375 0.038  0.0385 0.039  0.0395
 0.04   0.0405 0.041  0.0415 0.042  0.0425 0.043  0.0435 0.044  0.0445
 0.045  0.0455 0.046  0.0465 0.047  0.0475 0.048  0.0485 0.049  0.0495
 0.05   0.0505 0.051  0.0515 0.052  0.0525 0.053  0.0535 0.054  0.0545
 0.055  0.0555 0.056  0.0565 0.057  0.0575 0.058  0.0585 0.059  0.0595
 0.06   0.0605 0.061  0.0615 0.062  0.0625 0.063  0.0635 0.064  0.0645
 0.065  0.0655 0.066  0.0665 0.067  0.0675 0.068  0.0685 0.069  0.0695
 0.07 

/usr/local/lib/python3.10/dist-packages/scipy/integrate/_odepack_py.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


Streaming output truncated to the last 5000 lines.
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584324301e-05
-1.3888555584

In [ ]:
len(element_stress1)
#element_stress1[1].shape
#evals.tocsr()
#sparse.save_npz(fname, evals, compressed=True)
# np.savez(fname, evals=evals,evecs=evecs)

# npzfile = np.load(fname+'.npz')
# evals=npzfile['evals']
# evecs=npzfile['evecs']

603

In [ ]:
def error(al,al1):
  error=0
  for j in range(al.shape[0]-1):
   error=max(abs(al[j]-al1[j]),error)
  #print(al.shape,al1.shape)
  # for i in range(al.shape[1]-1):
  #   for j in range(al.shape[0]-1):
  #       #print(al[j][i]-al1[2*j][i])
  #       error[i]=max(abs((al[j][i]-al1[2*j][i])),error[i])
  #print(error)
  return error

In [ ]:
for ls in range(len(filenames)):
  plot_stress_xx.append(abs(element_stress1[ls][:,0]))
  plot_stress_yy.append(abs(element_stress1[ls][:,1]))
  plot_stress_xy.append(abs(element_stress1[ls][:,2]))
  plt.figure()
  #print(er)
vocabulary = [1 , 2 , 3, 0]
my_colors = {1:'red',2:'green',3:'blue'}
#dict_keys(['-', '--', '-.', ':', 'None', ' ', ''])
my_linestyles={ 1: '-',2:'--',3:':'}
er=-1
for ls in range(len(filenames)):
  #print(ls)
  if ls!=0:
    #er=np.max(plot_stress_xx[ls]-plot_stress_xx[ls-1])
    er=error(plot_stress_xx[ls],plot_stress_xx[ls-1]),er
    print(er)
    #er=error()
    
  plt.title("stress_xx")
  plt.plot(t,plot_stress_xx[ls],color=my_colors.get(vocabulary[ls], 'black'),linestyle=my_linestyles.get(vocabulary[ls],'-.'),label=filenames[ls])
  plt.legend()
print("xx error: {}".format(er))
#print(er)
plt.savefig('./stress_xx.png')
plt.close('all')
plt.figure()
er=0
for ls in range(len(filenames)):
  if ls!=0:
    er=error(plot_stress_yy[ls],plot_stress_yy[ls-1])
    print(er)
  plt.title("stress_yy")
  plt.plot(t,plot_stress_yy[ls],label=filenames[ls],color=my_colors.get(vocabulary[ls], 'black'),linestyle=my_linestyles.get(vocabulary[ls],'-.'))
  plt.legend()
plt.savefig('./stress_yy.png')
plt.close('all')
print("yy error: {}".format(er))
print(er)
er=0
plt.figure()
for ls in range(len(filenames)):
  if ls!=0:
    er=error(plot_stress_xy[ls],plot_stress_xy[ls-1])
    print(er)
  plt.title("stress_xy")
  plt.plot(t,plot_stress_xy[ls],label=filenames[ls],color=my_colors.get(vocabulary[ls], 'black'),linestyle=my_linestyles.get(vocabulary[ls],'-.'))
  plt.legend()
print("xy error: {}".format(er))
print(er)
plt.savefig('./stress_xy.png')
plt.close('all')

(4003.331713382802, -1)
(4086.4308077491933, (4003.331713382802, -1))
xx error: (4086.4308077491933, (4003.331713382802, -1))
5660.619968371279
22157.058771320222
yy error: 22157.058771320222
22157.058771320222
2413.3860302452813
4953.332149897683
xy error: 4953.332149897683
4953.332149897683


In [ ]:
print(ALPHA[:,j].shape,t.shape,ALPHA1[:,j].shape)

(101,) (101,) (201,)


In [ ]:
def error1(al,al1):
  error=np.zeros(al.shape[1])
  #for j in range(al.shape[0]-1):
   # error=max(abs((al[j]-al1[2*j])/al[j]),error)
  #print(al.shape,al1.shape)
  for i in range(al.shape[1]-1):
    for j in range(al.shape[0]-1):
        #print(al[j][i]-al1[2*j][i])
        error[i]=max(abs((al[j][i]-al1[2*j][i])),error[i])

  return error

In [ ]:

er1=np.zeros(n)
er1=error1(ALPHA,ALPHA1)
print(max(er1))

0.00035961930129352706


In [ ]:
#stress, deformations = GetP(el, ver, U, E, nu)

In [ ]:
from google.colab import files
import shutil
for fname in filenames:
  shutil.make_archive(fname, 'zip', fname)
  files.download(fname+".zip")


NameError: ignored